# To get access to the dataset:
- If you already have the folder with the datasets, you might need to 'git pull' to ensure that it is updated
> Else, clone repo using the command below <br>
> "git clone https://github.com/clemnyan/ENGS_108_Fall_2021.git" <br>


## **Problem 2: Introduction to TensorFlow**
In this problem, we will start working in tensorflow to build deep learning systems starting with fully connected neural networks. We will focus on using the food image dataset we built in the last problem.
>
> **(a)** Using the food image dataset we built in the last problem (last week's assignment!), build a [tensorflow Data Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) that is shuffled with a batch size of 10. 

In [ ]:
# Code and explanation

> **(b)** Build a two layer fully connected neural network of any size with a ReLu activation function and a final softmax layer.

In [ ]:
# Code and explanation

> **(c)** Compile your model with an appropriate loss function and optimizer. Briefly describe your choices.

In [ ]:
# Code and explanation 

> **(d)** Train your model on the food image training dataset. And report your accuracy on the testing dataset.

In [ ]:
# Code and explanation

> **(e)** Now try to tune this network by varying the number of layers, units, activations and see if you can outperform the network in part (d). Does your best model perform better or worse than the SVM in problem 1?

In [ ]:
# Code and explanation

> **(BONUS)** We lost a lot of information when we resized the images in part (a). What would happen if we didn't resize the images and we built fit the neural network with all this other information? Try it out! *Hint: Runtime will be much longer, both to create the image dataset without resizing and to train the model, so you might have to get the code working and then just let it run.*

In [7]:
# Code and explanation

> **(BONUS)** Implement and explain other feature engineering (and data augmentation) techniques that we can perform to increase prediction accuracy? 

In [8]:
# Code and explanation